<details>
<summary>Table of Contents</summary>

- [1. Connecting to CUDA](#1-connecting-to-cuda)
- [2. Informer](#2-informer)
- [3. PatchTST](#3-patchtst)

</details>



Script with Informer and PatchTST (default parameters).

In [21]:
# Import libraries
import torch
import pandas as pd
import numpy as np
import subprocess
import os
import time
import shutil

from utils.helper import extract_metrics_from_output, running_time

# 1. Connecting to CUDA

Run on GPU, because running it on CPU will cost a lot of time.


I do not recommend to run it in Google Colab, because it interrupts training process.

If you are not going to use remote servers with multiple GPUs, skip this part.

In [22]:
# For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [23]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [24]:
torch.cuda.get_device_name(1)

'Quadro RTX 6000'

In [25]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
cuda_device = "1"

# 2. Informer

In [17]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/informer/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [19]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]
seq_len = 96
model = "Informer"
itr = 2

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.0001
#n_heads = 16
e_layers = 2
d_layers = 1
loss = "MAE"

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results = []

In [26]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --d_layers {d_layers} \
              --factor 5 \
              --enc_in {num_cols[i]} \
              --dec_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --loss_fnc "{loss}" \
              --scaler_type minmax \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                informer_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5, learning_rate=0.000

ValueError: I/O operation on closed file.

In [28]:
# DFs per iteration
# Convert the collected data into DataFrame
informer_df_itrs = pd.DataFrame(informer_results)

# Set multi-index 
informer_df_itrs.set_index(['Country', 'Pred_len', 'Iteration'], inplace=True)
path = 'results/informer'

if not os.path.exists(path):
    os.makedirs(path)

informer_df_itrs.to_csv(os.path.join(path, 'informer_df_itrs.csv'))
informer_df_itrs.round(4)

MSE    RMSE     MAE
Country Pred_len Iteration                        
DE      24       1          0.0255  0.1597  0.1043
                 2          0.0271  0.1647  0.1041
        96       1          0.0411  0.2028  0.1419
                 2          0.0450  0.2121  0.1449
        168      1          0.0556  0.2359  0.1601
                 2          0.0512  0.2263  0.1570
GB      24       1          0.0337  0.1835  0.1234
                 2          0.0313  0.1769  0.1170
        96       1          0.0494  0.2222  0.1569
                 2          0.0518  0.2277  0.1581
        168      1          0.0553  0.2353  0.1653
                 2          0.0653  0.2556  0.1753
ES      24       1          0.0275  0.1657  0.0974
                 2          0.0193  0.1388  0.0870
        96       1          0.0488  0.2210  0.1398
                 2          0.0564  0.2376  0.1464
        168      1          0.0444  0.2107  0.1352
                 2          0.0602  0.2454  0.1583
FR      24       1          0.0125  0.1116  0.0628
                 2          0.0125  0.1118  0.0644
        96       1          0.0229  0.1512  0.0903
                 2          0.0221  0.1486  0.0892
        168      1          0.0243  0.1560  0.0966
                 2          0.0243  0.1559  0.0977
IT      24       1          0.0120  0.1097  0.0642
                 2          0.0131  0.1144  0.0672
        96       1          0.0223  0.1494  0.0913
                 2          0.0193  0.1388  0.0861
        168      1          0.0208  0.1442  0.0944
                 2          0.0234  0.1530  0.0991

In [30]:
# Final DF
informer_df = informer_df_itrs.groupby(['Country', 'Pred_len']).mean()
informer_df.columns = pd.MultiIndex.from_product([['Informer'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
informer_df.to_csv(os.path.join(path, 'informer.csv'))
informer_df.round(4)

Model            Informer                
Metrics               MSE    RMSE     MAE
Country Pred_len                         
DE      24         0.0263  0.1622  0.1042
        96         0.0430  0.2074  0.1434
        168        0.0534  0.2311  0.1585
ES      24         0.0234  0.1523  0.0922
        96         0.0526  0.2293  0.1431
        168        0.0523  0.2281  0.1467
FR      24         0.0125  0.1117  0.0636
        96         0.0225  0.1499  0.0897
        168        0.0243  0.1559  0.0971
GB      24         0.0325  0.1802  0.1202
        96         0.0506  0.2249  0.1575
        168        0.0603  0.2454  0.1703
IT      24         0.0126  0.1120  0.0657
        96         0.0208  0.1441  0.0887
        168        0.0221  0.1486  0.0967

# 3. PatchTST

We separated PatchTST from Informer, because it has additional arguments. It is not so easy to modify f-string (as e. g. distionary) to unpack some arguments with if statement. Moreover, it has different parameter values.

In [31]:
log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 336
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
patch_len = 32
stride = 16
batch_size = 32 # default from informer

# List to store the results
patchtst_results = []

In [32]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len {patch_len} \
              --stride {stride} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5, learning_rate=0.0

In [39]:
# DFs per iteration
# Convert the collected data into DataFrame
patchtst_df_itrs = pd.DataFrame(patchtst_results)

# Set multi-index 
patchtst_df_itrs.set_index(['Country', 'Pred_len', 'Iteration'], inplace=True)
path = 'results/patchtst'

if not os.path.exists(path):
    os.makedirs(path)

patchtst_df_itrs.to_csv(os.path.join(path, 'patchtst_df_itrs.csv'))
patchtst_df_itrs.round(4)

MSE    RMSE     MAE
Country Pred_len Iteration                        
DE      24       1          0.0212  0.1457  0.0880
                 2          0.0214  0.1462  0.0884
        96       1          0.0374  0.1935  0.1278
                 2          0.0372  0.1929  0.1278
        168      1          0.0396  0.1989  0.1348
                 2          0.0404  0.2011  0.1345
GB      24       1          0.0258  0.1607  0.1011
                 2          0.0257  0.1602  0.1008
        96       1          0.0436  0.2087  0.1420
                 2          0.0421  0.2051  0.1400
        168      1          0.0460  0.2146  0.1487
                 2          0.0471  0.2169  0.1489
ES      24       1          0.0098  0.0990  0.0592
                 2          0.0098  0.0992  0.0594
        96       1          0.0189  0.1373  0.0882
                 2          0.0189  0.1376  0.0877
        168      1          0.0215  0.1465  0.0942
                 2          0.0214  0.1464  0.0945
FR      24       1          0.0103  0.1014  0.0553
                 2          0.0104  0.1021  0.0556
        96       1          0.0192  0.1387  0.0815
                 2          0.0195  0.1395  0.0819
        168      1          0.0214  0.1461  0.0878
                 2          0.0212  0.1455  0.0873
IT      24       1          0.0102  0.1008  0.0564
                 2          0.0102  0.1009  0.0565
        96       1          0.0187  0.1368  0.0808
                 2          0.0190  0.1378  0.0810
        168      1          0.0211  0.1453  0.0873
                 2          0.0209  0.1446  0.0878

In [40]:
# Final DF
patchtst_df = patchtst_df_itrs.groupby(['Country', 'Pred_len']).mean()
patchtst_df.columns = pd.MultiIndex.from_product([['PatchTST'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst.csv'))
patchtst_df.round(4)

Model            PatchTST                
Metrics               MSE    RMSE     MAE
Country Pred_len                         
DE      24         0.0213  0.1460  0.0882
        96         0.0373  0.1932  0.1278
        168        0.0400  0.2000  0.1347
ES      24         0.0098  0.0991  0.0593
        96         0.0189  0.1374  0.0880
        168        0.0215  0.1465  0.0943
FR      24         0.0104  0.1018  0.0554
        96         0.0193  0.1391  0.0817
        168        0.0213  0.1458  0.0876
GB      24         0.0257  0.1605  0.1010
        96         0.0428  0.2069  0.1410
        168        0.0466  0.2158  0.1488
IT      24         0.0102  0.1008  0.0564
        96         0.0188  0.1373  0.0809
        168        0.0210  0.1450  0.0876

# OLD

In [ ]:
with open(log_file_path, "w") as log_file:

  for i, country in enumerate(countries):

    log_file.write(f"\n=== Starting experiments for country: {country} ===\n")

    for pred_len in pred_lens:
      model_id = f"{country}_{seq_len}_{pred_len}"
      dataset = f"{country}_data.csv"

      # Arguments for the command
      command = f"""
      python {script_path} \
        --random_seed 2021 \
        --is_training 1 \
        --root_path "{data_path}" \
        --data_path "{dataset}" \
        --model_id {model_id} \
        --model "{model}" \
        --data "custom" \
        --features M \
        --seq_len {seq_len} \
        --label_len 5 \
        --pred_len {pred_len} \
        --e_layers 3 \
        --factor 5 \
        --enc_in {num_cols[i]} \
        --dec_in {num_cols[i]} \
        --c_out {num_cols[i]} \
        --des 'Exp' \
        --train_epochs 20 \
        --patience 3 \
        --n_heads {n_heads} \
        --patch_len 32 \
        --stride 16 \
        --overlapping_windows \
        --itr 1 --batch_size 64 --learning_rate {lr}
      """
      # Log the country and prediction length
      log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

      # Run the command and capture the output
      process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

      # Capture the output in real-time
      output = []
      for line in process.stdout:
          output.append(line)
          print(line, end='')  # Print in the .ipynb cell
          log_file.write(line)  # Write to the log file

      # Wait for the process to complete
      process.wait()

      # Extract metrics from the captured output
      mse, mae = extract_metrics_from_output(output)

      # Log the extracted metrics
      log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
      log_file.write(f"MSE: {mse}, MAE: {mae}\n")

      # Append the results to the list
      patchtst_results.append({
          'Country': country,
          'Pred_len': pred_len,
          'MSE': mse,
          'MAE': mae
      })

In [ ]:
# Convert the collected data into DataFrame
patchtst_df = pd.DataFrame(patchtst_results)

# Set multi-index 
patchtst_df.set_index(['Country', 'Pred_len'], inplace=True)
patchtst_df = patchtst_df.round(4)